Pten wt - mutated 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import sys
sys.path.append('C:\\Users\\brittany henderson\\GitHub\\WhenMutationsDontMatter\\')
import plot_utils as p

In [2]:
gbm = cptac.Gbm()
endo = cptac.Endometrial()

In [3]:
cancer1 = "GBM"
cancer2 = "Endometrial"
gene = 'PTEN'

In [4]:
#GBM ttest
# Step 1 - Create dataframe in order to do comparisons with wrap_ttest
prot = gbm.get_proteomics()
gbm_prot_list = list(prot.columns)
prot_and_mutations = gbm.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = gbm_prot_list)  

# Step 2 - Create binary column 
prot_and_mutations['binary_mutations'] = np.where(
            prot_and_mutations[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype', 'Mutated')

# Step 3 - Format the dataframe correctly for the T-test(just omics and binary columns for tumors)
tumors = prot_and_mutations[prot_and_mutations.Sample_Status == 'Tumor'] #drop Normal samples
columns_to_drop = [gene+"_Mutation", gene+"_Location", gene+"_Mutation_Status", "Sample_Status"]
mut_status = tumors.drop(columns_to_drop, axis = 1)
mut_status = mut_status.dropna(axis=1,how='all')
gbm_prot = list(mut_status.columns[:-1]) # new prot list without na in all tumor (sometimes values in normal)

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 82 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 6)


Name,A1BG_proteomics,A2M_proteomics,AAAS_proteomics,AACS_proteomics,AADAT_proteomics,AAED1_proteomics,AAGAB_proteomics,AAK1_proteomics,AAMDC_proteomics,AAMP_proteomics,...,ZSWIM8_proteomics,ZW10_proteomics,ZWILCH_proteomics,ZWINT_proteomics,ZXDC_proteomics,ZYG11B_proteomics,ZYX_proteomics,ZZEF1_proteomics,ZZZ3_proteomics,binary_mutations
Sample_ID,,,,,,,,,,,,,,,,,,,,,
S001,0.077630,0.487228,-0.254208,-0.144373,0.551881,-0.025276,-0.467451,-0.089511,-0.078806,0.329603,...,-0.047437,-0.105908,-0.347076,NaN,0.459635,0.079452,-0.784983,-0.488441,0.167990,Mutated
S002,-0.145975,0.798796,0.184242,-0.470603,NaN,0.390211,0.245466,-0.609998,0.118625,-0.086927,...,0.161975,-0.213093,0.235571,NaN,0.107421,0.048724,0.138403,-0.290141,0.405037,Wildtype
S003,0.821991,1.096470,-0.094421,-0.106304,0.084578,0.176402,-0.248151,0.014061,-0.699773,-0.638462,...,-0.065534,-0.306717,0.879991,NaN,0.883564,-0.172222,0.011876,-0.131889,-0.503581,Mutated
S004,-0.064567,0.129385,0.047751,-0.118187,0.237434,NaN,0.303847,0.322163,-0.555479,-0.363414,...,-0.254535,0.463653,0.580230,0.503044,-0.604986,0.178077,-0.720059,-0.150197,-0.268715,Mutated
S005,-0.763691,-1.031834,-0.217194,-0.695701,0.184173,-0.474816,-0.051789,0.344842,-0.642746,0.068863,...,-0.092502,0.010639,-0.465079,NaN,-0.500083,0.112651,1.004660,-0.230304,-0.102416,Wildtype
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S100,0.230843,0.426624,0.420155,0.303960,0.078341,0.833128,0.205727,-0.434020,0.151202,0.208972,...,-0.021466,-0.049569,0.128461,-0.437797,-0.144343,-0.024981,-0.275897,-0.116367,0.351479,Mutated
S101,-0.635316,-0.478657,0.647004,-0.050322,-0.440517,0.092256,-0.006924,-0.511331,0.101877,0.128163,...,-0.034616,0.158269,NaN,NaN,0.906358,0.287317,0.745880,-0.320613,-0.180511,Wildtype
S102,0.616640,0.767029,-0.231897,0.050841,0.304359,-0.384608,-0.205284,0.513575,0.151769,-0.307037,...,-0.110846,-0.069605,-0.183921,NaN,-0.681170,0.095601,-0.096716,0.049811,-0.409252,Wildtype


In [5]:
mut_df = mut_status.loc[mut_status['binary_mutations'] == 'Mutated']
wt_df = mut_status.loc[mut_status['binary_mutations'] == 'Wildtype']

mut_med = mut_df.median()
wt_med = wt_df.median()

gbm_d = {}

for prot in gbm_prot:
    dif_gbm = mut_med[prot] - wt_med[prot]
    gbm_d[prot] = dif_gbm

gbm_df = pd.DataFrame.from_dict(gbm_d, orient='index', columns=['Gbm'])
gbm_df

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\lib\nanfunctions.py:1076: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [ ]:
# CHECK
gene = 'A2M'
dif_gbm = mut_med[gene+'_proteomics'] - wt_med[gene+'_proteomics']
print(mut_med[gene+'_proteomics'],' - ', wt_med[gene+'_proteomics'], dif_gbm)

Endo

In [7]:
#Endo ttest
# Step 1 - Create dataframe in order to do comparisons with wrap_ttest
prot = endo.get_proteomics()
en_prot_list = list(prot.columns)
prot_and_mutations = endo.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = en_prot_list)  

# Step 2 - Create binary column 
prot_and_mutations['binary_mutations'] = np.where(
            prot_and_mutations[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype', 'Mutated')

# Step 3 - Format the dataframe correctly for the T-test(just omics and binary columns for tumors)
tumors = prot_and_mutations[prot_and_mutations.Sample_Status == 'Tumor'] #drop Normal samples
columns_to_drop = [gene+"_Mutation", gene+"_Location", gene+"_Mutation_Status", "Sample_Status"]
mut_status = tumors.drop(columns_to_drop, axis = 1)
mut_status = mut_status.dropna(axis=1,how='all')
en_prot = list(mut_status.columns[:-1])

mut_status

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 69 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 6)


Name,A1BG_proteomics,A2M_proteomics,A2ML1_proteomics,A4GALT_proteomics,AAAS_proteomics,AACS_proteomics,AADAT_proteomics,AAED1_proteomics,AAGAB_proteomics,AAK1_proteomics,...,ZSWIM9_proteomics,ZW10_proteomics,ZWILCH_proteomics,ZWINT_proteomics,ZXDC_proteomics,ZYG11B_proteomics,ZYX_proteomics,ZZEF1_proteomics,ZZZ3_proteomics,binary_mutations
Sample_ID,,,,,,,,,,,,,,,,,,,,,
S001,-1.180,-0.8630,-0.8020,0.2220,0.2560,0.6650,1.2800,-0.3390,0.4120,-0.664,...,NaN,0.02290,0.1090,NaN,-0.332,-0.43300,-1.020,-0.12300,-0.0859,Mutated
S002,-0.685,-1.0700,-0.6840,0.9840,0.1350,0.3340,1.3000,0.1390,1.3300,-0.367,...,NaN,0.36300,1.0700,0.7370,-0.564,-0.00461,-1.130,-0.07570,-0.4730,Mutated
S003,-0.528,-1.3200,0.4350,NaN,-0.2400,1.0400,-0.0213,-0.0479,0.4190,-0.500,...,-0.1450,0.01050,-0.1160,NaN,0.151,-0.07400,-0.540,0.32000,-0.4190,Mutated
S005,-1.670,-1.1900,-0.4430,0.2430,-0.0993,0.7570,0.7400,-0.9290,0.2290,-0.223,...,-0.0552,-0.07140,0.0933,0.1560,-0.398,-0.07520,-0.797,-0.03010,-0.4670,Mutated
S006,-0.374,-0.0206,-0.5370,0.3110,0.3750,0.0131,-1.1000,NaN,0.5650,-0.101,...,NaN,-1.22000,-0.5620,0.9370,-0.646,0.20700,-1.850,-0.17600,0.0513,Wildtype
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S099,-1.070,-0.7120,0.0462,-0.0471,0.2270,1.3500,1.2100,0.0048,0.6820,-0.224,...,-0.4010,0.55700,0.8270,0.3480,-0.187,-0.10700,-0.830,0.06200,-0.5280,Mutated
S100,-1.280,-0.7360,-0.5520,0.1140,0.2730,1.0700,0.8500,NaN,-0.5360,-0.266,...,NaN,-0.00364,0.7630,0.0784,-0.203,0.01320,-1.200,-0.35700,0.0299,Wildtype
S101,-0.290,-0.3200,3.1700,-0.9070,0.0317,-0.0425,NaN,-0.2730,-0.0747,0.124,...,-0.5210,0.05230,0.5150,0.5850,0.138,-0.19000,-0.966,-0.00627,-0.2490,Wildtype


In [8]:
mut_df = mut_status.loc[mut_status['binary_mutations'] == 'Mutated']
wt_df = mut_status.loc[mut_status['binary_mutations'] == 'Wildtype']

mut_med = mut_df.median()
wt_med = wt_df.median()

en_d = {}

for prot in en_prot:
    dif_en = mut_med[prot] - wt_med[prot]
    en_d[prot] = dif_en

{'A1BG_proteomics': -0.08550000000000002,
 'A2M_proteomics': 0.01150000000000001,
 'A2ML1_proteomics': 0.53575,
 'A4GALT_proteomics': 0.403,
 'AAAS_proteomics': -0.24410000000000004,
 'AACS_proteomics': 0.3468,
 'AADAT_proteomics': -0.028999999999999998,
 'AAED1_proteomics': 0.0892,
 'AAGAB_proteomics': 0.17835,
 'AAK1_proteomics': -0.07390000000000001,
 'AAMDC_proteomics': 0.186,
 'AAMP_proteomics': -0.35250000000000004,
 'AAR2_proteomics': -0.14835,
 'AARS_proteomics': -0.1694,
 'AARS2_proteomics': -0.2294,
 'AARSD1_proteomics': 0.14303000000000002,
 'AASDHPPT_proteomics': -0.05154999999999999,
 'AASS_proteomics': 0.23450000000000004,
 'AATF_proteomics': -0.3553,
 'ABAT_proteomics': 0.23500000000000001,
 'ABCA8_proteomics': -0.056999999999999995,
 'ABCB1_proteomics': 0.2345,
 'ABCB10_proteomics': -0.08030000000000001,
 'ABCB6_proteomics': 0.23170000000000002,
 'ABCB7_proteomics': 0.005500000000000005,
 'ABCB8_proteomics': 0.1682,
 'ABCC1_proteomics': 0.155795,
 'ABCC10_proteomics': -

In [9]:
en_df = pd.DataFrame.from_dict(en_d, orient='index', columns=['Endo'])
en_df

,Endo
A1BG_proteomics,-0.08550
A2M_proteomics,0.01150
A2ML1_proteomics,0.53575
A4GALT_proteomics,0.40300
AAAS_proteomics,-0.24410
...,...
ZXDC_proteomics,-0.48150
ZYG11B_proteomics,-0.20160
ZYX_proteomics,0.08750
ZZEF1_proteomics,0.09025


In [20]:
n = gbm_df.join(en_df)
m = n.transpose()
m

,A1BG_proteomics,A2M_proteomics,AAAS_proteomics,AACS_proteomics,AADAT_proteomics,AAED1_proteomics,AAGAB_proteomics,AAK1_proteomics,AAMDC_proteomics,AAMP_proteomics,...,ZSCAN31_proteomics,ZSWIM8_proteomics,ZW10_proteomics,ZWILCH_proteomics,ZWINT_proteomics,ZXDC_proteomics,ZYG11B_proteomics,ZYX_proteomics,ZZEF1_proteomics,ZZZ3_proteomics
Gbm,0.040188,0.108524,0.058545,-0.057941,0.175684,0.063785,-0.036972,-0.069487,-0.144826,-0.07142,...,-0.096138,-0.049499,0.102503,0.158887,-0.246501,0.061036,0.064513,0.116899,0.03176,-0.004215
Endo,-0.085500,0.011500,-0.244100,0.346800,-0.029000,0.089200,0.178350,-0.073900,0.186000,-0.35250,...,0.212950,0.105550,0.056850,0.013500,-0.154000,-0.481500,-0.201600,0.087500,0.09025,-0.064000


Ovarian

In [23]:
ov = cptac.Ovarian()

In [28]:
# Step 1 - Create dataframe in order to do comparisons with wrap_ttest
prot = ov.get_proteomics()
ov_prot_list = list(prot.columns)
prot_and_mutations = ov.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = ov_prot_list) 
#prot_and_mutations = ov.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)
prot

ValueError: Names should be list-like for a MultiIndex

In [25]:
# Step 2 - Create binary column 
prot_and_mutations['binary_mutations'] = np.where(
            prot_and_mutations[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype', 'Mutated')

# Step 3 - Format the dataframe correctly for the T-test(just omics and binary columns for tumors)
tumors = prot_and_mutations[prot_and_mutations.Sample_Status == 'Tumor'] #drop Normal samples
columns_to_drop = [gene+"_Mutation", gene+"_Location", gene+"_Mutation_Status", "Sample_Status"]
mut_status = tumors.drop(columns_to_drop, axis = 1)
mut_status = mut_status.dropna(axis=1,how='all')
ov_prot = list(mut_status.columns[:-1])

ValueError: Names should be list-like for a MultiIndex

In [ ]:
mut_df = mut_status.loc[mut_status['binary_mutations'] == 'Mutated']
wt_df = mut_status.loc[mut_status['binary_mutations'] == 'Wildtype']

mut_med = mut_df.median()
wt_med = wt_df.median()

ov_dict = {}

for prot in ov_prot:
    delta_ov = mut_med[prot] - wt_med[prot]
    ov_dict[prot] = delta_ov

ov_df = pd.DataFrame.from_dict(ov_d, orient='index', columns=['Ovarian'])
ov_df

Breast

In [ ]:
cptac.download(dataset='brca', version='3.1')
b = cptac.Brca()

In [ ]:
# Step 1 - Create dataframe in order to do comparisons with wrap_ttest
prot = b.get_proteomics()
prot_list = list(prot.columns)
prot_and_mutations = b.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = prot_list) 
prot_and_mutations = b.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)

# Step 2 - Create binary column 
prot_and_mutations['binary_mutations'] = np.where(
            prot_and_mutations[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype', 'Mutated')

# Step 3 - Format the dataframe correctly for the T-test(just omics and binary columns for tumors)
tumors = prot_and_mutations[prot_and_mutations.Sample_Status == 'Tumor'] #drop Normal samples
columns_to_drop = [gene+"_Mutation", gene+"_Location", gene+"_Mutation_Status", "Sample_Status"]
mut_status = tumors.drop(columns_to_drop, axis = 1)
mut_status = mut_status.dropna(axis=1,how='all')
b_prot = list(mut_status.columns[:-1])

In [ ]:
mut_df = mut_status.loc[mut_status['binary_mutations'] == 'Mutated']
wt_df = mut_status.loc[mut_status['binary_mutations'] == 'Wildtype']

mut_med = mut_df.median()
wt_med = wt_df.median()

b_dict = {}

for prot in b_prot:
    delta_b = mut_med[prot] - wt_med[prot]
    b_d[prot] = delta_b

b_df = pd.DataFrame.from_dict(gbm_d, orient='index', columns=['Breast'])
b_df

Colon

In [29]:
col = cptac.Colon()

In [32]:
# Step 1 - Create dataframe in order to do comparisons with wrap_ttest
prot = col.get_proteomics()
col_prot_list = list(prot.columns)
prot_and_mutations = col.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = col_prot_list) 
#prot_and_mutations = col.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)

# Step 2 - Create binary column 
prot_and_mutations['binary_mutations'] = np.where(
            prot_and_mutations[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype', 'Mutated')

# Step 3 - Format the dataframe correctly for the T-test(just omics and binary columns for tumors)
tumors = prot_and_mutations[prot_and_mutations.Sample_Status == 'Tumor'] #drop Normal samples
columns_to_drop = [gene+"_Mutation", gene+"_Location", gene+"_Mutation_Status", "Sample_Status"]
mut_status = tumors.drop(columns_to_drop, axis = 1)
mut_status = mut_status.dropna(axis=1,how='all')
col_prot = list(mut_status.columns[:-1])

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 201 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 5)
cptac warning: proteomics data was not found for the following samples, so proteomics data columns were filled with NaN for these samples: S001, S013, S015, S017, S038, S080, S081, S090, S099, S109 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 5)


In [33]:
mut_df = mut_status.loc[mut_status['binary_mutations'] == 'Mutated']
wt_df = mut_status.loc[mut_status['binary_mutations'] == 'Wildtype']

mut_med = mut_df.median()
wt_med = wt_df.median()

col_dict = {}

for prot in col_prot:
    delta_col = mut_med[prot] - wt_med[prot]
    col_dict[prot] = delta_col

col_df = pd.DataFrame.from_dict(col_dict, orient='index', columns=['Colon'])
col_df

,Colon
A1BG_proteomics,-0.10650
A1CF_proteomics,-0.32690
A2M_proteomics,-0.12400
AAAS_proteomics,0.10800
AACS_proteomics,0.02100
...,...
ZW10_proteomics,0.06040
ZWILCH_proteomics,0.34465
ZWINT_proteomics,0.28480
ZYX_proteomics,-0.08650


Kidney

In [35]:
r = cptac.Ccrcc()
# SAME PROBLEM AS BREAST on join_omics_to_mut
#ValueError: Names should be list-like for a MultiIndex

In [37]:
# Step 1 - Create dataframe in order to do comparisons with wrap_ttest
prot = r.get_proteomics()
k_prot_list = list(prot.columns)
prot_and_mutations = r.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = k_prot_list) 
prot_and_mutations = r.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)

# Step 2 - Create binary column 
prot_and_mutations['binary_mutations'] = np.where(
            prot_and_mutations[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype', 'Mutated')

# Step 3 - Format the dataframe correctly for the T-test(just omics and binary columns for tumors)
tumors = prot_and_mutations[prot_and_mutations.Sample_Status == 'Tumor'] #drop Normal samples
columns_to_drop = [gene+"_Mutation", gene+"_Location", gene+"_Mutation_Status", "Sample_Status"]
mut_status = tumors.drop(columns_to_drop, axis = 1)
mut_status = mut_status.dropna(axis=1,how='all')
k_prot = list(mut_status.columns[:-1])

ValueError: Names should be list-like for a MultiIndex

In [ ]:
mut_df = mut_status.loc[mut_status['binary_mutations'] == 'Mutated']
wt_df = mut_status.loc[mut_status['binary_mutations'] == 'Wildtype']

mut_med = mut_df.median()
wt_med = wt_df.median()

k_dict = {}

for prot in k_prot:
    delta_k = mut_med[prot] - wt_med[prot]
    k_dict[prot] = delta_k

k_df = pd.DataFrame.from_dict(k_dict, orient='index', columns=['Kidney'])
k_df

Lung

In [ ]:
#cptac.download(dataset='luad')
l = cptac.Luad()

In [ ]:
l.list_data()

In [ ]:
# Step 1 - Create dataframe in order to do comparisons with wrap_ttest
prot = l.get_proteomics()
prot_list = list(prot.columns)
prot_and_mutations = l.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = prot_list) 
prot_and_mutations = l.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)

# Step 2 - Create binary column 
prot_and_mutations['binary_mutations'] = np.where(
            prot_and_mutations[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype', 'Mutated')

# Step 3 - Format the dataframe correctly for the T-test(just omics and binary columns for tumors)
tumors = prot_and_mutations[prot_and_mutations.Sample_Status == 'Tumor'] #drop Normal samples
columns_to_drop = [gene+"_Mutation", gene+"_Location", gene+"_Mutation_Status", "Sample_Status"]
mut_status = tumors.drop(columns_to_drop, axis = 1)
mut_status = mut_status.dropna(axis=1,how='all')
k_prot = list(mut_status.columns[:-1])

In [ ]:
mut_df = mut_status.loc[mut_status['binary_mutations'] == 'Mutated']
wt_df = mut_status.loc[mut_status['binary_mutations'] == 'Wildtype']

mut_med = mut_df.median()
wt_med = wt_df.median()

gbm_d = {}

for prot in gbm_prot:
    dif_gbm = mut_med[prot] - wt_med[prot]
    gbm_d[prot] = dif_gbm

gbm_df = pd.DataFrame.from_dict(gbm_d, orient='index', columns=['Gbm'])
gbm_df

Head and Neck

In [39]:
h = cptac.Hnscc()
#cptac.download(dataset='hnscc', version='0.1')

In [40]:
# Step 1 - Create dataframe in order to do comparisons with wrap_ttest
prot = h.get_proteomics()
h_prot_list = list(prot.columns)
prot_and_mutations = h.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = h_prot_list) 
#prot_and_mutations = r.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)

# Step 2 - Create binary column 
prot_and_mutations['binary_mutations'] = np.where(
            prot_and_mutations[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype', 'Mutated')

# Step 3 - Format the dataframe correctly for the T-test(just omics and binary columns for tumors)
tumors = prot_and_mutations[prot_and_mutations.Sample_Status == 'Tumor'] #drop Normal samples
columns_to_drop = [gene+"_Mutation", gene+"_Location", gene+"_Mutation_Status", "Sample_Status"]
mut_status = tumors.drop(columns_to_drop, axis = 1)
mut_status = mut_status.dropna(axis=1,how='all')
h_prot = list(mut_status.columns[:-1])

InvalidParameterError: PTEN gene not found in somatic_mutation data.

In [ ]:
mut_df = mut_status.loc[mut_status['binary_mutations'] == 'Mutated']
wt_df = mut_status.loc[mut_status['binary_mutations'] == 'Wildtype']

mut_med = mut_df.median()
wt_med = wt_df.median()

h_dict = {}

for prot in h_prot:
    dif_h = mut_med[prot] - wt_med[prot]
    h_dict[prot] = dif_h

h_df = pd.DataFrame.from_dict(h_dict, orient='index', columns=['Hnscc'])
h_df

Combine

In [ ]:
dd = {'Gbm': [dif_gbm],'Hscc':['add'],'Lung':['add'],'Breast':['add'],'Endo': [dif_en],
      'Ovarian': [dif_ov], 'Colon':[dif_colon],'Kidney':[dif_k] } #list, avoid from_dict error passing in scalars -index

pan_cancer_df = pd.DataFrame.from_dict(dd, orient='index', columns=['Directionality']) #orient makes keys index
pan_cancer_df

In [41]:
n = gbm_df.join(en_df)
m = n.join(col_df)
o = m.transpose()
o

,A1BG_proteomics,A2M_proteomics,AAAS_proteomics,AACS_proteomics,AADAT_proteomics,AAED1_proteomics,AAGAB_proteomics,AAK1_proteomics,AAMDC_proteomics,AAMP_proteomics,...,ZSCAN31_proteomics,ZSWIM8_proteomics,ZW10_proteomics,ZWILCH_proteomics,ZWINT_proteomics,ZXDC_proteomics,ZYG11B_proteomics,ZYX_proteomics,ZZEF1_proteomics,ZZZ3_proteomics
Gbm,0.040188,0.108524,0.058545,-0.057941,0.175684,0.063785,-0.036972,-0.069487,-0.144826,-0.07142,...,-0.096138,-0.049499,0.102503,0.158887,-0.246501,0.061036,0.064513,0.116899,0.03176,-0.004215
Endo,-0.085500,0.011500,-0.244100,0.346800,-0.029000,0.089200,0.178350,-0.073900,0.186000,-0.35250,...,0.212950,0.105550,0.056850,0.013500,-0.154000,-0.481500,-0.201600,0.087500,0.09025,-0.064000
Colon,-0.106500,-0.124000,0.108000,0.021000,NaN,NaN,0.450500,0.105800,-0.101000,0.01775,...,NaN,NaN,0.060400,0.344650,0.284800,NaN,NaN,-0.086500,-0.01700,NaN


In [45]:
s = o.std()
s

A1BG_proteomics      0.079326
A2M_proteomics       0.116792
AAAS_proteomics      0.190619
AACS_proteomics      0.214551
AADAT_proteomics     0.144734
                       ...   
ZXDC_proteomics      0.383631
ZYG11B_proteomics    0.188171
ZYX_proteomics       0.109933
ZZEF1_proteomics     0.053699
ZZZ3_proteomics      0.042274
Length: 11141, dtype: float64